In [1]:
import os
from functools import partial
from datetime import datetime
import re

from pyspark import SparkContext

In [2]:
index2events_column = {
  0: "GlobalEventId",
  37: "Actor1Geo_CountryCode",
  60: "url",
  40: "lat",
  41: "long",
  59: "DateAdded"
}

index2mentions_column = {
  0: "GlobalEventId",
  3: "MentionType",
  5: "MentionIdentifier",
  1: "EventTimeDate",
  2: "MentionTimeDate"
}

def convert_to_datetime(datetime_str):
    return datetime.strptime(datetime_str, "%Y%m%d%H%M%S")

print(convert_to_datetime("20200326190000"))

type_converters = {
    "EventTimeDate":convert_to_datetime,
    "MentionTimeDate":convert_to_datetime,
    "DateAdded": convert_to_datetime
}

2020-03-26 19:00:00


In [3]:
sc = SparkContext()

In [4]:
basepath = "/home/ubuntu/data/project/gdelt"
mentions_path_english = os.path.join(basepath,"english/mentions/")
mentions_path_multi = os.path.join(basepath,"multilingual/mentions/")
events_path_english = os.path.join(basepath,"english/events/")
events_path_multi = os.path.join(basepath,"multilingual/events/")



In [5]:
print(mentions_path_english)

/home/ubuntu/data/project/gdelt/english/mentions/


In [6]:
mentions_english_rdd = sc.textFile(mentions_path_english)
mentions_multi_rdd = sc.textFile(mentions_path_multi)
mentions_rdd = mentions_english_rdd.union(mentions_multi_rdd)

In [7]:
for line in mentions_rdd.take(10):
    print(line)

914887164	20200327113000	20200327140000	1	brownsvilleherald.com	https://www.brownsvilleherald.com/news/us_world/political-turmoil-in-kabul-dogs-negotiations-with-taliban/article_48bc7cd9-f4e3-5c34-979b-16f8e4e0cec4.html	7	-1	1240	1232	1	30	3138	-5.94059405940594		
914887164	20200327113000	20200327140000	1	570news.com	https://www.570news.com/2020/03/27/political-turmoil-in-kabul-dogs-negotiations-with-taliban/	7	-1	1223	1215	1	30	3147	-5.71992110453649		
914887168	20200327113000	20200327140000	1	brownsvilleherald.com	https://www.brownsvilleherald.com/news/us_world/political-turmoil-in-kabul-dogs-negotiations-with-taliban/article_48bc7cd9-f4e3-5c34-979b-16f8e4e0cec4.html	7	1206	1235	1227	0	20	3138	-5.94059405940594		
914887168	20200327113000	20200327140000	1	570news.com	https://www.570news.com/2020/03/27/political-turmoil-in-kabul-dogs-negotiations-with-taliban/	7	1189	1218	1210	0	20	3147	-5.71992110453649		
914887169	20200327113000	20200327140000	1	brownsvilleherald.com	https://www.brow

In [8]:
mentions_rdd = mentions_rdd.map(lambda line: line.split("\t"))

In [9]:
print(mentions_rdd.take(1)[0])

['914887164', '20200327113000', '20200327140000', '1', 'brownsvilleherald.com', 'https://www.brownsvilleherald.com/news/us_world/political-turmoil-in-kabul-dogs-negotiations-with-taliban/article_48bc7cd9-f4e3-5c34-979b-16f8e4e0cec4.html', '7', '-1', '1240', '1232', '1', '30', '3138', '-5.94059405940594', '', '']


In [10]:
def transform_to_json(record, index2colname):
    return {colname: record[index] for index, colname in index2colname.items() }

mentions_json_rdd = mentions_rdd.map(partial(transform_to_json,index2colname=index2mentions_column))

In [11]:
mentions_json_rdd.take(1)

[{'GlobalEventId': '914887164',
  'MentionType': '1',
  'MentionIdentifier': 'https://www.brownsvilleherald.com/news/us_world/political-turmoil-in-kabul-dogs-negotiations-with-taliban/article_48bc7cd9-f4e3-5c34-979b-16f8e4e0cec4.html',
  'EventTimeDate': '20200327113000',
  'MentionTimeDate': '20200327140000'}]

In [12]:
def convert_values(record):
    for key, value in record.items():
        try:
            record[key] = type_converters[key](value)
        except KeyError:
            continue
    return record

In [13]:
mentions_converted_rdd = mentions_json_rdd.filter(lambda record: record!=None).map(convert_values)

In [14]:
mentions_converted_rdd.take(1)

[{'GlobalEventId': '914887164',
  'MentionType': '1',
  'MentionIdentifier': 'https://www.brownsvilleherald.com/news/us_world/political-turmoil-in-kabul-dogs-negotiations-with-taliban/article_48bc7cd9-f4e3-5c34-979b-16f8e4e0cec4.html',
  'EventTimeDate': datetime.datetime(2020, 3, 27, 11, 30),
  'MentionTimeDate': datetime.datetime(2020, 3, 27, 14, 0)}]

In [15]:
mentions_web_rdd = mentions_converted_rdd.filter(lambda record: record["MentionType"]=="1")

In [16]:
print(mentions_web_rdd.take(1))

[{'GlobalEventId': '914887164', 'MentionType': '1', 'MentionIdentifier': 'https://www.brownsvilleherald.com/news/us_world/political-turmoil-in-kabul-dogs-negotiations-with-taliban/article_48bc7cd9-f4e3-5c34-979b-16f8e4e0cec4.html', 'EventTimeDate': datetime.datetime(2020, 3, 27, 11, 30), 'MentionTimeDate': datetime.datetime(2020, 3, 27, 14, 0)}]


In [17]:
event_mentions_count_rdd = (mentions_web_rdd
                            .map(lambda record: (record["GlobalEventId"],1))
                            .reduceByKey(lambda x, y: x+y)
                           )

In [18]:
event_mentions_count_rdd.take(10)

[('914911553', 1),
 ('914798373', 48),
 ('914911657', 1),
 ('914795977', 110),
 ('914802390', 5),
 ('914911951', 1),
 ('914817118', 1),
 ('914795291', 23),
 ('914814306', 2),
 ('914912463', 1)]

In [19]:
events_english_rdd = sc.textFile(events_path_english)
events_multi_rdd = sc.textFile(events_path_multi)
events_rdd = events_english_rdd.union(events_multi_rdd)

In [20]:
events_rdd = events_rdd.map(lambda line: line.split("\t"))

In [21]:
events_json_rdd = events_rdd.map(partial(transform_to_json,index2colname=index2events_column))

In [22]:
print(events_json_rdd.take(1))

[{'GlobalEventId': '914971029', 'Actor1Geo_CountryCode': '', 'url': 'https://www.slashgear.com/gm-ventilator-plans-revealed-as-trump-backtracks-on-coronavirus-kit-27614681/', 'lat': '', 'long': '', 'DateAdded': '20200327200000'}]


In [23]:
corona_regex = re.compile("(?=.*virus)(?=.*corona)|[Cc][Oo][Vv][Ii][Dd]-?19")

In [24]:
events_corona_rdd = events_json_rdd.filter(lambda record: corona_regex.search(record["url"])!=None)

In [25]:
events_geo_rdd = (events_corona_rdd
                  .map(lambda record: (record["GlobalEventId"],(record["lat"],record["long"])))
                  .filter(lambda tpl: tpl[1]!=('','')))

In [26]:
events_geo_rdd.take(1)

[('914971030', ('-17.8178', '31.0447'))]

In [27]:
joined_rdd = events_geo_rdd.leftOuterJoin(event_mentions_count_rdd)

In [28]:
joined_rdd.take(2)

[('914971994', (('30.1669', '-84.3332'), 1)),
 ('914972519', (('42.1497', '-74.9384'), 1))]

In [29]:
import mysql.connector

In [34]:
def create_mysql_cursor(user, password, database):
    cnx = mysql.connector.connect(user=user, password=password, database=database)
    cursor = cnx.cursor()
    return cnx, cursor

mysql_cursor = partial(create_mysql_cursor, user="root", password="testtest",database="Corona")

In [ ]:
def store_partition(partition_iterator, connection_factory):
    user = "root"
    password = "testtest"
    database = "Corona"
    cnx = mysql.connector.connect(user=user, password=password, database=database)
    cursor = cnx.cursor()
    #cursor = connection_factory()
    
    records = [(event_id, float(lat), float(long), count) for event_id, ((lat, long), count) in partition_iterator]
    
    add_event = ("INSERT INTO GeoEvents "
               "(DateAdded, EventId, CountryCode, Latitude, Longitude, MentionCount) "
               "VALUES (%s, %s, %s, %s, %s, %s)")
    
    cursor.executemany(add_event, records)
    
    cnx.commit()
    cnx.close()

joined_rdd.foreachPartition(partial(store_partition, connection_factory=mysql_cursor))

In [35]:
cnx, cur = mysql_cursor()

In [36]:
it = cur.execute("select CountryCode, count(distinct EventId) as NumberOfEvents from GeoEvents2 where MentionCount>10 group by CountryCode order by NumberOfEvents desc;")

In [38]:
for i in cur:
    print(i)

('AU', 34)
('US', 16)
('DE', 16)
('ES', 10)
('GB', 10)
('CN', 7)
('PK', 6)
('', 6)
('IT', 5)
('CA', 3)
('IE', 1)
('IL', 1)
('KE', 1)
('NZ', 1)
